In [63]:
gwas_file = "/home/xutingfeng/GIFT/data/GWAS/T1Mapping_Cortex_20240129.csv_firstorder_Median_all_2023_GRCh38_unionKidneys.tsv.gz"

In [64]:
save_dir = "/home/xutingfeng/GIFT/data/analysis/DNAJC16/polyfun"
from pathlib import Path

Path(save_dir).mkdir(parents=True, exist_ok=True)

In [65]:
# from finemap_tools.reader.gwas import load_GWASFormated_file


# topLoci = (1, 15583355, 15583355)  # (chr, start, end)
# locus_range = 250  # 100kb
# locus_range = locus_range * 1000
# locus_range_tuple = (topLoci[0], topLoci[1] - locus_range, topLoci[2] + locus_range)

# locus_region = f"{locus_range_tuple[0]}:{locus_range_tuple[1]}-{locus_range_tuple[2]}"
# print(f"locus_region: {locus_region}")

In [55]:
default_polyfun_map = {
    "SNP": "SNP",
    "CHR": "chrom",
    "BP": "pos",
    "A1": "alt",  # A1 is the effect allele
    "A2": "ref",  # A2 is the other allele
    "Z": None,
    "SNPVAR": None,
}


def sumstats2polyfun(sumstats, map_dict=None):
    sumstats = sumstats.copy()
    polyfun_cols = list(default_polyfun_map.keys())
    used_polyfun_map_dict = default_polyfun_map.copy()
    if map_dict:
        used_polyfun_map_dict.update(map_dict)

    for col in polyfun_cols:
        current_check_col = used_polyfun_map_dict.get(col, None)

        if (current_check_col not in sumstats.columns) | (current_check_col is None):
            if col == "Z":
                # assert beta and se in map_dict
                assert (
                    "beta" in used_polyfun_map_dict.keys()
                    and "se" in used_polyfun_map_dict.keys()
                ), "beta and se should be in" + str(used_polyfun_map_dict)
                sumstats["Z"] = sumstats[map_dict["beta"]].astype(float) / sumstats[
                    map_dict["se"]
                ].astype(float)
                used_polyfun_map_dict["Z"] = "Z"
            elif col == "SNPVAR":
                if current_check_col not in sumstats.columns:
                    print(f"SNPVAR not in sumstats, will pass it")
                polyfun_cols.remove(col)
            else:
                raise ValueError(f"{current_check_col} not in sumstats")

    rename_dict = {v: k for k, v in used_polyfun_map_dict.items() if v is not None}
    sumstats.rename(columns=rename_dict, inplace=True)
    return sumstats[polyfun_cols]

In [77]:
from finemap_tools.reader.gwas import load_GWASFormated_file
from finemap_tools.utils import add_ID
from finemap_tools.snpfilter import filter_pipline

# f"1:14533350-16133356"
sumstats = load_GWASFormated_file(gwas_file, region=f"1")  # load

Will load region from file.
tabix have a header, so will take the first line as header and remove it.


/home/xutingfeng/github_code/mine/finemap_tools/finemap_tools/reader/reader.py:104: DtypeWarning: Columns (0,1,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(StringIO(stdout), sep="\t", header=None)


In [78]:
# sumstats = load_GWASFormated_file(gwas_file, region=locus_region)  # load
sumstats["SNP"] = add_ID(sumstats, ["chrom", "pos", "ref", "alt"])  # add rsid
# sumstats = filter_pipline(sumstats=sumstats, id_col="SNP")  # filter

In [79]:
polyfun_sumstats = sumstats2polyfun(sumstats, map_dict={"beta": "beta", "se": "sebeta"})
polyfun_sumstats

SNPVAR not in sumstats, will pass it


,SNP,CHR,BP,A1,A2,Z
1,1:10177:A:AC,1,10177,AC,A,-1.621657
2,1:10352:T:TA,1,10352,TA,T,0.734889
3,1:10616:CCGCCGTTGCAAAGGCGCGCCG:C,1,10616,C,CCGCCGTTGCAAAGGCGCGCCG,0.519880
4,1:11008:C:G,1,11008,G,C,-0.779716
5,1:11012:C:G,1,11012,G,C,-0.779716
...,...,...,...,...,...,...
1539599,1:248945900:T:TA,1,248945900,TA,T,-0.008871
1539600,1:248946020:A:T,1,248946020,T,A,-0.339775
1539601,1:248946338:GGT:G,1,248946338,G,GGT,1.832058
1539602,1:248946340:T:G,1,248946340,G,T,-1.493662


In [81]:
# polyfun_sumstats[polyfun_sumstats.duplicated(subset=["SNP"])]
polyfun_sumstats = polyfun_sumstats.drop_duplicates(subset=["BP"])

In [82]:
outdir = "/home/xutingfeng/GIFT/data/analysis/DNAJC16/polyfun"
polyfun_sumstats.to_csv(f"{outdir}/polyfun_sumstats.tsv", sep="\t", index=False)

In [62]:
polyfun_sumstats["CHR"] = polyfun_sumstats["CHR"].astype("str")
polyfun_sumstats["BP"] = polyfun_sumstats["BP"].astype("str")

polyfun_sumstats.to_parquet(f"{outdir}/polyfun_sumstats.parquet", index=False)

In [49]:
import os

bfile_path = "/mnt/d/桌面/work/GIFT/data/pgen/DNAJC16_GRCh38"

bgen_path = f"{bfile_path}.bgen"
sample_path = f"{bfile_path}.sample"
bgi_path = f"{bfile_path}.bgen.bgi"

tgt_bgen = os.path.join(save_dir, "data.bgen")
tgt_sample = os.path.join(save_dir, "data.sample")
tgt_bgi = os.path.join(save_dir, "data.bgen.bgi")
tgt_z = os.path.join(save_dir, "data.z")


# for bgen link data
try:
    os.symlink(bgen_path, tgt_bgen)
    os.symlink(sample_path, tgt_sample)
    os.symlink(bgi_path, tgt_bgi)
except:
    pass

In [2]:
a = {"3": 4}

In [4]:
vars(a)

TypeError: vars() argument must have __dict__ attribute